# PulsimCore High-Performance API

This notebook demonstrates the new API features:

1. Standalone device objects with CRTP architecture
2. Advanced convergence aids (Gmin stepping, source stepping, pseudo-transient)
3. BDF integration with adaptive timestep control
4. Analytical solutions for validation
5. SIMD detection and high-performance features
6. Benchmark framework

In [1]:
from pathlib import Path
import sys

_root = Path.cwd()
for _ in range(6):
    candidate = _root / 'build' / 'python'
    if candidate.is_dir():
        if str(candidate) not in sys.path:
            sys.path.insert(0, str(candidate))
        break
    _root = _root.parent

import pulsim as ps
import numpy as np
import matplotlib.pyplot as plt

print(f"API loaded successfully")

API loaded successfully


## 1. Device Objects

The API provides standalone device objects that can be configured independently.

In [2]:
# Create passive devices
r1 = ps.Resistor(1000.0, "R1")  # 1k Ohm
c1 = ps.Capacitor(1e-6, 0.0, "C1")  # 1uF, 0V initial
l1 = ps.Inductor(1e-3, 0.0, "L1")  # 1mH, 0A initial

# Create sources
vs = ps.VoltageSource(12.0, "Vdc")
cs = ps.CurrentSource(0.001, "I1")

print(f"R1: {r1.resistance()} Ohms")
print(f"C1: {c1.capacitance() * 1e6:.2f} uF")
print(f"L1: {l1.inductance() * 1e3:.2f} mH")
print(f"Vdc: {vs.voltage()} V")
print(f"I1: {cs.current() * 1e3:.2f} mA")

R1: 1000.0 Ohms
C1: 1.00 uF
L1: 1.00 mH
Vdc: 12.0 V
I1: 1.00 mA


## 2. Device Types and Solver Status

The API provides comprehensive enumerations.

In [3]:
# Device types
print("Device Types:")
for dtype in [ps.DeviceType.Resistor, ps.DeviceType.Capacitor, 
              ps.DeviceType.Inductor, ps.DeviceType.MOSFET]:
    print(f"  - {dtype}")

print("\nSolver Status Codes:")
for status in [ps.SolverStatus.Success, ps.SolverStatus.MaxIterationsReached,
               ps.SolverStatus.SingularMatrix, ps.SolverStatus.Diverging]:
    print(f"  - {ps.solver_status_to_string(status)}")

Device Types:
  - DeviceType.Resistor
  - DeviceType.Capacitor
  - DeviceType.Inductor
  - DeviceType.MOSFET

Solver Status Codes:
  - Success
  - MaxIterationsReached
  - SingularMatrix
  - Diverging


## 3. Solver Configuration

The API provides fine-grained control over solver behavior.

In [4]:
# Default tolerances
tols = ps.Tolerances.defaults()
print("Default Tolerances:")
print(f"  Voltage absolute: {tols.voltage_abstol:.2e}")
print(f"  Current absolute: {tols.current_abstol:.2e}")
print(f"  Residual: {tols.residual_tol:.2e}")

# Newton solver options
opts = ps.NewtonOptions()
print(f"\nNewton Options:")
print(f"  Max iterations: {opts.max_iterations}")
print(f"  Initial damping: {opts.initial_damping}")
print(f"  Auto damping: {opts.auto_damping}")

Default Tolerances:
  Voltage absolute: 1.00e-09
  Current absolute: 1.00e-12
  Residual: 1.00e-09

Newton Options:
  Max iterations: 50
  Initial damping: 1.0
  Auto damping: True


## 4. DC Convergence Aids

The API includes advanced strategies for difficult DC operating points.

In [5]:
# Gmin stepping configuration
gmin = ps.GminConfig()
print("Gmin Stepping:")
print(f"  Initial Gmin: {gmin.initial_gmin:.2e}")
print(f"  Final Gmin: {gmin.final_gmin:.2e}")
print(f"  Reduction factor: {gmin.reduction_factor}")
print(f"  Required steps: {gmin.required_steps()}")

# Source stepping configuration
src = ps.SourceSteppingConfig()
print(f"\nSource Stepping:")
print(f"  Initial scale: {src.initial_scale}")
print(f"  Final scale: {src.final_scale}")
print(f"  Max steps: {src.max_steps}")

# Pseudo-transient configuration
ptran = ps.PseudoTransientConfig()
print(f"\nPseudo-Transient:")
print(f"  Initial dt: {ptran.initial_dt:.2e}")
print(f"  Max dt: {ptran.max_dt:.2e}")

# DC convergence with auto strategy
dc_config = ps.DCConvergenceConfig()
print(f"\nDC Strategy: {dc_config.strategy}")

Gmin Stepping:
  Initial Gmin: 1.00e-02
  Final Gmin: 1.00e-12
  Reduction factor: 10.0
  Required steps: 10

Source Stepping:
  Initial scale: 0.0
  Final scale: 1.0
  Max steps: 100

Pseudo-Transient:
  Initial dt: 1.00e-09
  Max dt: 1.00e+03

DC Strategy: DCStrategy.Auto


## 5. Integration Methods

BDF order control and adaptive timestep with PI controller.

In [6]:
# BDF order configuration
bdf = ps.BDFOrderConfig()
print("BDF Order Control:")
print(f"  Min order: {bdf.min_order}")
print(f"  Max order: {bdf.max_order}")
print(f"  Initial order: {bdf.initial_order}")
print(f"  Auto order: {bdf.enable_auto_order}")

# Timestep configurations
print("\nTimestep Configurations:")
for name, cfg in [("Default", ps.TimestepConfig.defaults()),
                   ("Conservative", ps.TimestepConfig.conservative()),
                   ("Aggressive", ps.TimestepConfig.aggressive())]:
    print(f"  {name}:")
    print(f"    dt_min: {cfg.dt_min:.2e}, dt_max: {cfg.dt_max:.2e}")
    print(f"    safety_factor: {cfg.safety_factor:.2f}")

BDF Order Control:
  Min order: 1
  Max order: 5
  Initial order: 1
  Auto order: True

Timestep Configurations:
  Default:
    dt_min: 1.00e-15, dt_max: 1.00e-03
    safety_factor: 0.90
  Conservative:
    dt_min: 1.00e-15, dt_max: 1.00e-03
    safety_factor: 0.80
  Aggressive:
    dt_min: 1.00e-15, dt_max: 1.00e-03
    safety_factor: 0.95


## 6. Analytical Solutions for Validation

Built-in analytical solutions for RC, RL, and RLC circuits.

In [7]:
# RC circuit: 1k Ohm, 1uF, step from 0V to 5V
rc = ps.RCAnalytical(1000, 1e-6, 0.0, 5.0)
tau = rc.tau()

print(f"RC Circuit (R=1k, C=1uF):")
print(f"  Time constant: {tau * 1e3:.3f} ms")
print(f"  V(0): {rc.voltage(0):.3f} V")
print(f"  V(tau): {rc.voltage(tau):.3f} V (63.2%)")
print(f"  V(5*tau): {rc.voltage(5*tau):.3f} V (99.3%)")

# Plot RC response
t = np.linspace(0, 5*tau, 100)
v = [rc.voltage(ti) for ti in t]

plt.figure(figsize=(10, 5))
plt.plot(t * 1e3, v, 'b-', linewidth=2)
plt.axhline(y=5*0.632, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=tau*1e3, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('RC Circuit Step Response')
plt.grid(True, alpha=0.3)
plt.annotate(f'τ = {tau*1e3:.2f} ms', xy=(tau*1e3, 5*0.632), 
             xytext=(tau*1e3 + 0.5, 2.5), fontsize=10)
plt.show()

RC Circuit (R=1k, C=1uF):
  Time constant: 1.000 ms
  V(0): 0.000 V
  V(tau): 3.161 V (63.2%)
  V(5*tau): 4.966 V (99.3%)


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_50789/1012476369.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# RL circuit: 1k Ohm, 1mH, 10V source
rl = ps.RLAnalytical(1000, 1e-3, 10.0, 0.0)
tau_rl = rl.tau()

print(f"RL Circuit (R=1k, L=1mH, V=10V):")
print(f"  Time constant: {tau_rl * 1e6:.3f} us")
print(f"  Final current: {rl.I_final() * 1e3:.3f} mA")

# Plot RL response
t = np.linspace(0, 5*tau_rl, 100)
i = [rl.current(ti) * 1e3 for ti in t]

plt.figure(figsize=(10, 5))
plt.plot(t * 1e6, i, 'r-', linewidth=2)
plt.xlabel('Time (us)')
plt.ylabel('Current (mA)')
plt.title('RL Circuit Step Response')
plt.grid(True, alpha=0.3)
plt.show()

RL Circuit (R=1k, L=1mH, V=10V):
  Time constant: 1.000 us
  Final current: 10.000 mA


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_50789/1604531933.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
# RLC circuits with different damping
print("RLC Circuit Damping Analysis:\n")

# Underdamped: small R
rlc_under = ps.RLCAnalytical(10, 1e-3, 1e-6, 10.0, 0.0, 0.0)
print(f"Underdamped (R=10):")
print(f"  omega_0: {rlc_under.omega_0():.1f} rad/s")
print(f"  zeta: {rlc_under.zeta():.3f}")
print(f"  Damping: {rlc_under.damping_type()}")

# Overdamped: large R
rlc_over = ps.RLCAnalytical(1000, 1e-3, 1e-6, 10.0, 0.0, 0.0)
print(f"\nOverdamped (R=1000):")
print(f"  omega_0: {rlc_over.omega_0():.1f} rad/s")
print(f"  zeta: {rlc_over.zeta():.3f}")
print(f"  Damping: {rlc_over.damping_type()}")

# Plot underdamped response
t = np.linspace(0, 0.01, 500)
v_under = [rlc_under.voltage(ti) for ti in t]

plt.figure(figsize=(10, 5))
plt.plot(t * 1e3, v_under, 'g-', linewidth=2)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Underdamped RLC Step Response')
plt.grid(True, alpha=0.3)
plt.show()

RLC Circuit Damping Analysis:

Underdamped (R=10):
  omega_0: 31622.8 rad/s
  zeta: 0.158
  Damping: RLCDamping.Underdamped

Overdamped (R=1000):
  omega_0: 31622.8 rad/s
  zeta: 15.811
  Damping: RLCDamping.Overdamped


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_50789/384736573.py:28: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 7. Waveform Validation

Compare simulation results against analytical solutions.

In [10]:
# Generate analytical waveform
rc = ps.RCAnalytical(1000, 1e-6, 0.0, 5.0)
times = np.linspace(0, 5e-3, 50)

# Simulated data (with slight error for demo)
sim_data = [(t, rc.voltage(t) + np.random.normal(0, 0.001)) for t in times]
ana_data = [(t, rc.voltage(t)) for t in times]

# Compare waveforms
result = ps.compare_waveforms("RC_validation", sim_data, ana_data, 0.01)

print("Validation Result:")
print(f"  Test: {result.test_name}")
print(f"  Passed: {result.passed}")
print(f"  Max error: {result.max_error:.6f}")
print(f"  RMS error: {result.rms_error:.6f}")
print(f"  Points: {result.num_points}")

Validation Result:
  Test: RC_validation
  Passed: True
  Max error: 0.002697
  RMS error: 0.000894
  Points: 50


In [11]:
# Export validation results
csv_output = ps.export_validation_csv([result])
print("CSV Export:")
print(csv_output)



CSV Export:
test_name,passed,num_points,max_error,rms_error,max_relative_pct,threshold_pct
RC_validation,true,50,2.696709e-03,8.942901e-04,8.592859e-02,1.000000e+00



## 8. High-Performance Features

SIMD detection and linear solver configuration.

In [12]:
# SIMD detection
simd_level = ps.detect_simd_level()
vector_width = ps.simd_vector_width()

print("SIMD Capability:")
print(f"  Level: {simd_level}")
print(f"  Vector width: {vector_width} doubles")

# Linear solver configuration
ls_config = ps.LinearSolverConfig()
print(f"\nLinear Solver:")
print(f"  Pivot tolerance: {ls_config.pivot_tolerance:.2e}")
print(f"  Reuse symbolic: {ls_config.reuse_symbolic}")
print(f"  Deterministic pivoting: {ls_config.deterministic_pivoting}")

SIMD Capability:
  Level: SIMDLevel.NEON
  Vector width: 2 doubles

Linear Solver:
  Pivot tolerance: 1.00e-10
  Reuse symbolic: True
  Deterministic pivoting: True


## 9. Benchmark Framework

Tools for performance measurement and comparison.

In [13]:
# Create benchmark structures
timing = ps.BenchmarkTiming()
timing.name = "RC_transient"
timing.iterations = 1000

result = ps.BenchmarkResult()
result.circuit_name = "RC_filter"
result.num_nodes = 2
result.num_devices = 3
result.num_timesteps = 5000

print("Benchmark Result:")
print(f"  Circuit: {result.circuit_name}")
print(f"  Nodes: {result.num_nodes}")
print(f"  Devices: {result.num_devices}")
print(f"  Timesteps: {result.num_timesteps}")

Benchmark Result:
  Circuit: RC_filter
  Nodes: 2
  Devices: 3
  Timesteps: 5000


## Summary

The API provides:

- **Standalone device objects** with CRTP architecture for 2x+ performance
- **Advanced convergence aids** for difficult DC operating points
- **BDF integration** with automatic order control (1-2)
- **PI timestep controller** for adaptive simulation
- **Built-in validation** against analytical solutions
- **SIMD optimization** with runtime detection
- **Benchmark framework** for performance measurement

See the [Migration Guide](../../docs/migration-guide.md) for transitioning from v1.